In [17]:
%run dihiggs_dataset.ipynb

import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F   # NOTE: I don't think this is used
import torch.autograd as autograd
import torch

print(torch.cuda.is_available())

True


In [18]:

os.makedirs("images", exist_ok=True)

#parser = argparse.ArgumentParser()

"""
parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=28, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--n_critic", type=int, default=5, help="number of training steps for discriminator per iter")
parser.add_argument("--clip_value", type=float, default=0.01, help="lower and upper clip value for disc. weights")
parser.add_argument("--sample_interval", type=int, default=400, help="interval betwen image samples")
"""
#opt = parser.parse_args()
#print(opt)
class opt():   # Class used for optimizers in the future. Defines all variables and stuff needed.
    n_epochs = 20000   # an epoch is the number of times it works through the entire training set
    batch_size = 4000   # the training set is broken up into batches, 
                        # and the average loss is used from a given batch for back propagation
    lr = 0.001   # learning rate (how much to change based on error)
    b1 = 0.9   # Used for Adam. Exponential decay rate for the first moment
    b2 = 0.999   # Used for Adam. Exponential decay rate for the second moment estimates (gradient squared)
    #n_cpu = 2    not used rn
    latent_dim = 100 #size of noise input to generator (latent space) 
    #img_size = 28
    # channels = 1   # Only used for img_shape right below, and img_shape isn't needed
    n_critic = 5   # The generator is trained after this many critic steps
    #   clip_value = 0.01   # No other usages rn. 
    sample_interval = 400   # Determines when a to save the image(s?) generated

# img_shape = (opt.channels, opt.img_size, opt.img_size)   # Not used rn

cuda = True if torch.cuda.is_available() else False



class Generator(nn.Module):
    """
    Create hidden layers. Apply normalization. Apply leaky relu. 
    """
    def __init__(self):
        super(Generator, self).__init__()   

        def block(in_feat, out_feat, normalize=True):   # This function creates the hidden layers
            layers = [nn.Linear(in_feat, out_feat)]   # layer is a hidden layer. Takes input
                                                      # (batch_size,in_feat) and give an output (batch_size,out_feat)
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))   # adds normalization to what Layers does to input and comes out in
                                                               # size (batch_size,out_feat). I think this does bn1d(linear(input))
            layers.append(nn.LeakyReLU(0.2, inplace=True))   # inplace means just modify input, don't allocate more memory
            return layers


        self.model = nn.Sequential(   
            """
            stores layers and functions applied to layers
            """
            *block(opt.latent_dim, 128, normalize=False),   # first layer
            *block(128, 256),   # second layer
            *block(256, 512),   # 3rd layer
            *block(512, 1024),   # 4th layer
            nn.Linear(1024, 25),   # final layer. Output is size 25
            nn.Tanh()   # Using tanh for final output (why tanh vs leaky relu?)
        )

    def forward(self, z):
        """
        applies layers to input to get img
        """
        img = self.model(z)   # applies model (layers and functions on layers) to z
        #img = img.view(img.shape[0], *img_shape)
        return img


class Discriminator(nn.Module):
    """
    Discriminator/critic layers
    """
    def __init__(self):
        super(Discriminator, self).__init__()   # Just uses the module constructor with name Discriminator 

        self.model = nn.Sequential(
            nn.Linear(25, 512),   # first layer
            nn.LeakyReLU(0.2, inplace=True),   # apply leaky relu to layer
            nn.Linear(512, 256),   # 2nd layer
            nn.LeakyReLU(0.2, inplace=True),   # apply leaky relu to layer
            nn.Linear(256, 1),   # Final layer to give output. Output is size 1 (validity score)
                                 # NOTE: weird to end with comma
        )

    def forward(self, img):
        """
        applies model to image and gives validity score
        """
        img_flat = img.view(img.shape[0], -1)   
        validity = self.model(img_flat)   # calculates validity score
        return validity


# ******* OUT OF CLASSES NOW ************


# Loss weight for gradient penalty
lambda_gp = 10

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()

 

# Configure data loader - CHANGE
os.makedirs("./data/", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    DiHiggsSignalMCDataset('./DiHiggs Data', generator_level = False),
    batch_size=opt.batch_size,
    shuffle=True,
)
print('done')

done


In [ ]:
# Optimizers (Adam optimizers are an alternative to stochastic gradient descent. TODO learn more about them)
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))   
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


def compute_gradient_penalty(D, real_samples, fake_samples):   
    """
    Calculates the gradient penalty loss for WGAN GP
    D input will be discrimantor function
    real_samples and fake_samples are from reality and generator. Both are sent in via memory location of buffer
    
    """
    # Random weight term for interpolation between real and fake samples (how much of each)
    alpha = Tensor(np.random.random((real_samples.size(0),1)))   # This is a tensor designating which to use where

  #  print(alpha.shape)
    # Get random interpolation between real and fake samples
   # print(real_samples.shape)
    
    # Gets some of real and some of fake samples for gradient penalty calculation
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    # .requires grad is something attached to all tensors and can be used to speed up (by making false I think)
    # It is automatically false, but if you need gradient then set to be true

   # print(interpolates.shape)

    d_interpolates = D(interpolates)   # Run discriminator on interpolates to get validity scores
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)   
    
    # Get gradient w.r.t. interpolates (uses inputs and outputs to calculate )
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,  
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)   # still need to figure out view thing
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()   # Normalize and then average gradients for penalty
    return gradient_penalty


# ----------
#  Training
# ----------

batches_done = 0   # Counter for batches
for epoch in range(opt.n_epochs):   # Loop through all epochs
    for i, x in enumerate(dataloader): # x is in dataloader (a batch I think). i
                                       # is the index of x (number of times critic is trained this epoch)

        # Configure input
        real_imgs = Variable(x.type(Tensor))   # Variable is a wrapper for the Tensor x was just made into

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()   # Make gradients zero so they don't accumulate

        # Sample noise (latent space) to make generator input
        z = Variable(Tensor(np.random.normal(0, 1, (x.shape[0], opt.latent_dim))))   # Once again Variable wraps the Tensor
#         print(type(x))
#         print(x.shape)
#         print(x[0].shape())
#         print(z.shape)

        # Generate a batch of images from the latent space sampled
        fake_imgs = generator(z)

        #print(fake_imgs[0])

        # Calculate validity score for real images
        real_validity = discriminator(real_imgs)

        # Calculate validity score for fake images
        fake_validity = discriminator(fake_imgs)

        # Calculate gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)

        # Calculate loss for critic (Adversarial loss)
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

        d_loss.backward()   # Do back propagation 
        optimizer_D.step()   # Update parameters based on gradients for individuals

        optimizer_G.zero_grad()   # Resets gradients for generator to be zero to avoid accumulation

        # Train the generator every n_critic steps
        if i % opt.n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            # Generate a batch of images
            fake_imgs = generator(z)

            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)

            g_loss.backward()
            optimizer_G.step()


            # ----------------------------
            # Save stuff when time is right
            # ----------------------------
            if batches_done % 10 == 0:
                print(
                    "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                    % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
                )

            if batches_done % opt.sample_interval == 0:
                save_image(fake_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

            batches_done += opt.n_critic
    if epoch % 10 == 0:
        z = Variable(Tensor(np.random.normal(0, 1, (300000, opt.latent_dim))))
        fake_data = generator(z)
        np.save('./gen_data/{num_batches}.npy'.format(num_batches=batches_done), fake_data.cpu().detach().numpy())

#for i, x in enumerate(dataloader):
    #print(x[0])
#Evaluation KS metric
# Generate ~300,000 samples and save (NumPy array)
# Load ^, 5000 then
# KS <-- Root


[Epoch 0/20000] [Batch 0/48] [D loss: 7.807000] [G loss: 0.003080]
[Epoch 0/20000] [Batch 10/48] [D loss: 2.810584] [G loss: 0.005898]
[Epoch 0/20000] [Batch 20/48] [D loss: -0.222216] [G loss: 0.004025]
[Epoch 0/20000] [Batch 30/48] [D loss: -0.975447] [G loss: 0.001766]
[Epoch 0/20000] [Batch 40/48] [D loss: -1.372036] [G loss: -0.004986]
[Epoch 1/20000] [Batch 0/48] [D loss: -1.580358] [G loss: -0.032039]
[Epoch 1/20000] [Batch 10/48] [D loss: -1.771839] [G loss: -0.054470]
[Epoch 1/20000] [Batch 20/48] [D loss: -1.888384] [G loss: -0.091709]
[Epoch 1/20000] [Batch 30/48] [D loss: -1.898808] [G loss: -0.149305]
[Epoch 1/20000] [Batch 40/48] [D loss: -1.848319] [G loss: -0.230189]
[Epoch 2/20000] [Batch 0/48] [D loss: -1.730096] [G loss: -0.352243]
[Epoch 2/20000] [Batch 10/48] [D loss: -1.554174] [G loss: -0.500081]
[Epoch 2/20000] [Batch 20/48] [D loss: -1.337315] [G loss: -0.674442]
[Epoch 2/20000] [Batch 30/48] [D loss: -1.053313] [G loss: -0.877273]
[Epoch 2/20000] [Batch 40/48]

[Epoch 23/20000] [Batch 30/48] [D loss: -0.248397] [G loss: -0.601448]
[Epoch 23/20000] [Batch 40/48] [D loss: -0.160122] [G loss: -1.110771]
[Epoch 24/20000] [Batch 0/48] [D loss: -0.001911] [G loss: -1.394879]
[Epoch 24/20000] [Batch 10/48] [D loss: 0.094909] [G loss: -1.281476]
[Epoch 24/20000] [Batch 20/48] [D loss: -0.098710] [G loss: -0.081973]
[Epoch 24/20000] [Batch 30/48] [D loss: -0.309345] [G loss: 1.180143]
[Epoch 24/20000] [Batch 40/48] [D loss: -0.298122] [G loss: 1.601127]
[Epoch 25/20000] [Batch 0/48] [D loss: -0.155004] [G loss: 1.469430]
[Epoch 25/20000] [Batch 10/48] [D loss: -0.051211] [G loss: 0.983336]
[Epoch 25/20000] [Batch 20/48] [D loss: -0.097156] [G loss: 0.196125]
[Epoch 25/20000] [Batch 30/48] [D loss: -0.196970] [G loss: -0.514238]
[Epoch 25/20000] [Batch 40/48] [D loss: -0.182630] [G loss: -0.891965]
[Epoch 26/20000] [Batch 0/48] [D loss: -0.070038] [G loss: -1.073145]
[Epoch 26/20000] [Batch 10/48] [D loss: 0.042822] [G loss: -1.004559]
[Epoch 26/20000]

[Epoch 47/20000] [Batch 0/48] [D loss: -0.125383] [G loss: -0.669584]
[Epoch 47/20000] [Batch 10/48] [D loss: -0.163237] [G loss: -0.973164]
[Epoch 47/20000] [Batch 20/48] [D loss: -0.145378] [G loss: -0.713805]
[Epoch 47/20000] [Batch 30/48] [D loss: -0.151749] [G loss: 0.041562]
[Epoch 47/20000] [Batch 40/48] [D loss: -0.142329] [G loss: 0.907693]
[Epoch 48/20000] [Batch 0/48] [D loss: -0.080892] [G loss: 1.380287]
[Epoch 48/20000] [Batch 10/48] [D loss: -0.038570] [G loss: 1.650274]
[Epoch 48/20000] [Batch 20/48] [D loss: -0.028502] [G loss: 1.284647]
[Epoch 48/20000] [Batch 30/48] [D loss: -0.133355] [G loss: 0.246516]
[Epoch 48/20000] [Batch 40/48] [D loss: -0.249251] [G loss: -0.573629]
[Epoch 49/20000] [Batch 0/48] [D loss: -0.193328] [G loss: -0.825982]
[Epoch 49/20000] [Batch 10/48] [D loss: -0.115286] [G loss: -0.754515]
[Epoch 49/20000] [Batch 20/48] [D loss: -0.048072] [G loss: -0.349377]
[Epoch 49/20000] [Batch 30/48] [D loss: -0.027357] [G loss: 0.428806]
[Epoch 49/20000]

[Epoch 70/20000] [Batch 20/48] [D loss: -0.037777] [G loss: 0.160679]
[Epoch 70/20000] [Batch 30/48] [D loss: -0.142647] [G loss: 1.158044]
[Epoch 70/20000] [Batch 40/48] [D loss: -0.142270] [G loss: 1.362476]
[Epoch 71/20000] [Batch 0/48] [D loss: -0.080900] [G loss: 1.069563]
[Epoch 71/20000] [Batch 10/48] [D loss: -0.062277] [G loss: 0.357934]
[Epoch 71/20000] [Batch 20/48] [D loss: -0.113146] [G loss: -0.490613]
[Epoch 71/20000] [Batch 30/48] [D loss: -0.120518] [G loss: -0.974469]
[Epoch 71/20000] [Batch 40/48] [D loss: -0.059810] [G loss: -1.043310]
[Epoch 72/20000] [Batch 0/48] [D loss: 0.004979] [G loss: -0.753169]
[Epoch 72/20000] [Batch 10/48] [D loss: -0.031071] [G loss: 0.215485]
[Epoch 72/20000] [Batch 20/48] [D loss: -0.157561] [G loss: 1.426445]
[Epoch 72/20000] [Batch 30/48] [D loss: -0.184412] [G loss: 1.806800]
[Epoch 72/20000] [Batch 40/48] [D loss: -0.112008] [G loss: 1.521988]
[Epoch 73/20000] [Batch 0/48] [D loss: -0.020158] [G loss: 0.978961]
[Epoch 73/20000] [Ba

[Epoch 93/20000] [Batch 40/48] [D loss: -0.057446] [G loss: 0.341403]
[Epoch 94/20000] [Batch 0/48] [D loss: -0.083707] [G loss: -0.268345]
[Epoch 94/20000] [Batch 10/48] [D loss: -0.118363] [G loss: -0.817223]
[Epoch 94/20000] [Batch 20/48] [D loss: -0.102178] [G loss: -1.042495]
[Epoch 94/20000] [Batch 30/48] [D loss: -0.063690] [G loss: -0.931607]
[Epoch 94/20000] [Batch 40/48] [D loss: -0.021063] [G loss: -0.500535]
[Epoch 95/20000] [Batch 0/48] [D loss: 0.007248] [G loss: -0.007133]
[Epoch 95/20000] [Batch 10/48] [D loss: -0.046353] [G loss: 0.791274]
[Epoch 95/20000] [Batch 20/48] [D loss: -0.101310] [G loss: 1.218364]
[Epoch 95/20000] [Batch 30/48] [D loss: -0.112930] [G loss: 1.046263]
[Epoch 95/20000] [Batch 40/48] [D loss: -0.101560] [G loss: 0.519254]
[Epoch 96/20000] [Batch 0/48] [D loss: -0.082301] [G loss: -0.019885]
[Epoch 96/20000] [Batch 10/48] [D loss: -0.072517] [G loss: -0.561611]
[Epoch 96/20000] [Batch 20/48] [D loss: -0.068875] [G loss: -0.937209]
[Epoch 96/20000

...

[Epoch 2875/20000] [Batch 30/48] [D loss: 0.049149] [G loss: -1.037012]
[Epoch 2875/20000] [Batch 40/48] [D loss: -0.001653] [G loss: 0.172200]
[Epoch 2876/20000] [Batch 0/48] [D loss: -0.053962] [G loss: 1.175094]
[Epoch 2876/20000] [Batch 10/48] [D loss: -0.097294] [G loss: 1.852725]
[Epoch 2876/20000] [Batch 20/48] [D loss: -0.103716] [G loss: 2.015779]
[Epoch 2876/20000] [Batch 30/48] [D loss: -0.074655] [G loss: 1.974682]
[Epoch 2876/20000] [Batch 40/48] [D loss: -0.069652] [G loss: 1.691800]
[Epoch 2877/20000] [Batch 0/48] [D loss: -0.031744] [G loss: 1.460377]
[Epoch 2877/20000] [Batch 10/48] [D loss: -0.015226] [G loss: 1.112706]
[Epoch 2877/20000] [Batch 20/48] [D loss: 0.002721] [G loss: 0.602173]
[Epoch 2877/20000] [Batch 30/48] [D loss: -0.001166] [G loss: -0.106929]
[Epoch 2877/20000] [Batch 40/48] [D loss: -0.007543] [G loss: -1.085586]
[Epoch 2878/20000] [Batch 0/48] [D loss: -0.024681] [G loss: -1.958662]
[Epoch 2878/20000] [Batch 10/48] [D loss: -0.039667] [G loss: -2.

[Epoch 2898/20000] [Batch 20/48] [D loss: -0.019615] [G loss: 0.697360]
[Epoch 2898/20000] [Batch 30/48] [D loss: -0.010658] [G loss: 0.535929]
[Epoch 2898/20000] [Batch 40/48] [D loss: 0.001332] [G loss: 0.280531]
[Epoch 2899/20000] [Batch 0/48] [D loss: 0.012988] [G loss: -0.050628]
[Epoch 2899/20000] [Batch 10/48] [D loss: 0.010592] [G loss: -0.643732]
[Epoch 2899/20000] [Batch 20/48] [D loss: -0.007873] [G loss: -1.465350]
[Epoch 2899/20000] [Batch 30/48] [D loss: -0.028133] [G loss: -2.297694]
[Epoch 2899/20000] [Batch 40/48] [D loss: -0.048040] [G loss: -2.923912]
[Epoch 2900/20000] [Batch 0/48] [D loss: 0.006668] [G loss: -3.343972]
[Epoch 2900/20000] [Batch 10/48] [D loss: -0.032570] [G loss: -3.319673]
[Epoch 2900/20000] [Batch 20/48] [D loss: -0.021865] [G loss: -3.397166]
[Epoch 2900/20000] [Batch 30/48] [D loss: -0.012873] [G loss: -3.393929]
[Epoch 2900/20000] [Batch 40/48] [D loss: -0.003282] [G loss: -3.319074]
[Epoch 2901/20000] [Batch 0/48] [D loss: 0.005656] [G loss: 

[Epoch 2921/20000] [Batch 10/48] [D loss: 0.046680] [G loss: 0.843699]
[Epoch 2921/20000] [Batch 20/48] [D loss: 0.036100] [G loss: 0.653029]
[Epoch 2921/20000] [Batch 30/48] [D loss: 0.007725] [G loss: 0.192188]
[Epoch 2921/20000] [Batch 40/48] [D loss: -0.046579] [G loss: -0.511634]
[Epoch 2922/20000] [Batch 0/48] [D loss: -0.074538] [G loss: -1.004459]
[Epoch 2922/20000] [Batch 10/48] [D loss: -0.082985] [G loss: -1.330272]
[Epoch 2922/20000] [Batch 20/48] [D loss: -0.076933] [G loss: -1.372445]
[Epoch 2922/20000] [Batch 30/48] [D loss: -0.064142] [G loss: -1.356156]
[Epoch 2922/20000] [Batch 40/48] [D loss: -0.032995] [G loss: -1.183422]
[Epoch 2923/20000] [Batch 0/48] [D loss: -0.006119] [G loss: -0.972267]
[Epoch 2923/20000] [Batch 10/48] [D loss: -0.004218] [G loss: -0.580284]
[Epoch 2923/20000] [Batch 20/48] [D loss: 0.002394] [G loss: 0.015700]
[Epoch 2923/20000] [Batch 30/48] [D loss: -0.009025] [G loss: 0.710527]
[Epoch 2923/20000] [Batch 40/48] [D loss: -0.017887] [G loss: 

[Epoch 2944/20000] [Batch 0/48] [D loss: 0.100067] [G loss: 0.657174]
[Epoch 2944/20000] [Batch 10/48] [D loss: 0.075678] [G loss: 1.248738]
[Epoch 2944/20000] [Batch 20/48] [D loss: 0.020753] [G loss: 1.885504]
[Epoch 2944/20000] [Batch 30/48] [D loss: -0.038255] [G loss: 2.002748]
[Epoch 2944/20000] [Batch 40/48] [D loss: -0.072094] [G loss: 1.671273]
[Epoch 2945/20000] [Batch 0/48] [D loss: -0.063043] [G loss: 1.262954]
[Epoch 2945/20000] [Batch 10/48] [D loss: -0.074904] [G loss: 0.779319]
[Epoch 2945/20000] [Batch 20/48] [D loss: -0.067180] [G loss: 0.157741]
[Epoch 2945/20000] [Batch 30/48] [D loss: -0.053060] [G loss: -0.537150]
[Epoch 2945/20000] [Batch 40/48] [D loss: -0.055794] [G loss: -1.318253]
[Epoch 2946/20000] [Batch 0/48] [D loss: -0.044898] [G loss: -2.006748]
[Epoch 2946/20000] [Batch 10/48] [D loss: -0.057250] [G loss: -2.778569]
[Epoch 2946/20000] [Batch 20/48] [D loss: -0.058960] [G loss: -3.380882]
[Epoch 2946/20000] [Batch 30/48] [D loss: -0.051887] [G loss: -3.

[Epoch 2966/20000] [Batch 40/48] [D loss: -0.021365] [G loss: -2.800002]
[Epoch 2967/20000] [Batch 0/48] [D loss: -0.013324] [G loss: -2.198606]
[Epoch 2967/20000] [Batch 10/48] [D loss: -0.021056] [G loss: -1.509245]
[Epoch 2967/20000] [Batch 20/48] [D loss: -0.013108] [G loss: -0.932271]
[Epoch 2967/20000] [Batch 30/48] [D loss: -0.000226] [G loss: -0.476065]
[Epoch 2967/20000] [Batch 40/48] [D loss: 0.008539] [G loss: -0.146379]
[Epoch 2968/20000] [Batch 0/48] [D loss: 0.026726] [G loss: 0.037441]
[Epoch 2968/20000] [Batch 10/48] [D loss: 0.029219] [G loss: 0.106698]
[Epoch 2968/20000] [Batch 20/48] [D loss: 0.016263] [G loss: -0.048668]
[Epoch 2968/20000] [Batch 30/48] [D loss: -0.021602] [G loss: -0.491154]
[Epoch 2968/20000] [Batch 40/48] [D loss: -0.047930] [G loss: -0.888089]
[Epoch 2969/20000] [Batch 0/48] [D loss: -0.017034] [G loss: -1.165920]
[Epoch 2969/20000] [Batch 10/48] [D loss: -0.051480] [G loss: -1.046481]
[Epoch 2969/20000] [Batch 20/48] [D loss: -0.046723] [G loss